# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

experiment_name = 'capstoneproject'

experiment=Experiment(ws, experiment_name)

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'final-compute'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found the compute cluster


In [3]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart_failure_clinical_records_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("dataset found!")

else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

dataset found!
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy", "experiment_timeout_minutes":30, "enable_early_stopping":True, "n_cross_validations":3,"max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name = 'DEATH_EVENT',**automl_settings)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on final-compute with default configuration
Running on remote compute: final-compute
Parent Run ID: AutoML_00d20164-fd7d-4ffe-af4f-085a8b0b3d25

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing value

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_00d20164-fd7d-4ffe-af4f-085a8b0b3d25',
 'target': 'final-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-19T04:48:34.902214Z',
 'endTimeUtc': '2021-01-19T05:05:53.780455Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'final-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"capstoneproject","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-135301","workspace_name":"quick-starts-ws-135301","region":"southcentralus","compute_target":"final-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: capstoneproject,
Id: AutoML_00d20164-fd7d-4ffe-af4f-085a8b0b3d25_40,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_lambda=1.0416666666666667,
                                                                                               scale_pos_weight=1,
                      

In [11]:
best_run_metrics

{'f1_score_micro': 0.8748348348348348,
 'f1_score_macro': 0.8536422791741941,
 'matthews_correlation': 0.710548460533944,
 'average_precision_score_micro': 0.9199739365528713,
 'AUC_weighted': 0.9034889586537783,
 'precision_score_weighted': 0.8736809756546599,
 'recall_score_micro': 0.8748348348348348,
 'AUC_macro': 0.9034889586537783,
 'log_loss': 0.3838258860670112,
 'recall_score_weighted': 0.8748348348348348,
 'f1_score_weighted': 0.8728708114240029,
 'precision_score_macro': 0.8681767185056658,
 'weighted_accuracy': 0.898723173507561,
 'average_precision_score_weighted': 0.9128802763486711,
 'average_precision_score_macro': 0.8938275077587697,
 'accuracy': 0.8748348348348348,
 'balanced_accuracy': 0.8428914238803412,
 'precision_score_micro': 0.8748348348348348,
 'recall_score_macro': 0.8428914238803412,
 'norm_macro_recall': 0.6857828477606823,
 'AUC_micro': 0.9212086031977925,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_00d20164-fd7d-4ffe-af4f-085a8b0b3d25_40

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#TODO: Save the best model
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

In [ ]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [ ]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script="scoring.py",environment=myenv)

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="heart-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
print(service.get_logs())

2021-01-19T00:50:57,776027517+00:00 - rsyslog/run 
2021-01-19T00:50:57,776144220+00:00 - gunicorn/run 
2021-01-19T00:50:57,776145420+00:00 - iot-server/run 
2021-01-19T00:50:57,827470300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [17]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/score

swagger URI: 
http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/swagger.json

http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/score
http://ec7be877-6f29-47ac-aab9-cbdf6f72b95f.southcentralus.azurecontainer.io/swagger.json


In [18]:
import requests
import json

data={"data":
  [{'age': 87.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 981,
  'diabetes': 0,
  'ejection_fraction': 55,
  'high_blood_pressure': 1,
  'platelets': 368000,
  'serum_creatinine': 3.5,
  'serum_sodium': 137,
  'sex': 0,
  'smoking': 0,
  'time': 4}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

The output is:  Must pass 2-d input


In [19]:
result.status_code

200

In [20]:
service.delete()
compute_target.delete()
model.delete()
run.delete()
experiment.delete()

NameError: name 'run' is not defined